In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 23.3 MB/s eta 0:00:00


In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("phatjk/bloomz-lora-vi-QA-NLLB-viquad")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to(device)
model = PeftModel.from_pretrained(model, "phatjk/bloomz-lora-vi-QA-NLLB-viquad")

In [ ]:
config

PeftConfig(peft_type='LORA', base_model_name_or_path='bigscience/bloomz-1b1', revision=None, task_type='CAUSAL_LM', inference_mode=True)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

In [ ]:
question = "Hãy cho tôi ba mẹo để giữ sức khỏe."
prompt = "Hỏi:"+ question + '''
    Đáp:'''
prompt

'Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.\n    Đáp:'

In [ ]:
import torch
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=2,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: 1 Thực hành chế độ ăn uống lành mạnh Ăn nhiều trái cây rau quả và ngũ cốc nguyên hạt để cung cấp chất dinh dưỡng cần thiết cho cơ thể 2 Tập thể dục thường xuyên Hoạt động thể chất có thể giúp giảm nguy cơ mắc bệnh mãn tính như tiểu đường, béo phì và bệnh tim mạch 3 Giảm căng thẳng và lo lắng Tập trung vào các hoạt động nhẹ nhàng như thiền, yoga hoặc chơi nhạc nhẹ để giảm mức độ căng cơ bắp và tăng mức năng lượng tổng thể 
Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: 1 Thực hành chế độ ăn uống lành mạnh Ăn nhiều trái cây rau quả và ngũ cốc nguyên hạt để cung cấp chất dinh dưỡng cần thiết cho cơ thể 2 Tập thể dục thường xuyên Hoạt động thể chất có thể giúp giảm nguy cơ mắc bệnh mãn tính như tiểu đường, béo phì và bệnh tim mạch 3 Giảm căng thẳng và lo lắng Tập trung vào các hoạt động nhẹ nhàng như thiền, yoga hoặc chơi nhạc nhẹ để giảm mức độ căng cơ bắp và tăng mức năng lượng tâm thần 
Hỏi:Hãy cho tôi ba mẹo để giữ sức khỏe.
    Đáp: 1

In [ ]:
question2 = "Ai là họa sĩ nổi tiếng nhất thế giới?"
prompt = "Hỏi:"+ question2 + '''
    Đáp:'''
prompt
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=3,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Ai là họa sĩ nổi tiếng nhất thế giới?
    Đáp: Pablo Picasso là một họa sĩ người Tây Ban Nha nổi tiếng với các tác phẩm của mình được biết đến rộng rãi trên toàn thế giới. Ông là một trong những họa sĩ có ảnh hưởng nhất trong lịch sử nghệ thuật thế giới và là một tác phẩm nghệ thuật nổi tiếng của thế kỷ 20. Ông cũng là một nghệ sĩ có tầm ảnh hưởng lớn đối với các nghệ sĩ khác trong thế giới nghệ thuật. Ông đã tạo ra một số trong số những tác phẩm vĩ đại nhất trong nghệ thuật
Hỏi:Ai là họa sĩ nổi tiếng nhất thế giới?
    Đáp: Pablo Picasso là một họa sĩ người Tây Ban Nha nổi tiếng với các tác phẩm của mình được biết đến rộng rãi trên toàn thế giới. Ông là một trong những họa sĩ có ảnh hưởng nhất trong lịch sử nghệ thuật thế giới và là một tác phẩm nghệ thuật nổi tiếng của thế kỷ 20. Ông cũng là một nghệ sĩ có tầm ảnh hưởng lớn đối với các nghệ sĩ khác trong thế giới nghệ thuật. Ông đã tạo ra một số trong số những tác phẩm vĩ đại nhất của thế giới
Hỏi:Ai là họa sĩ nổi tiếng nhất thế 

In [ ]:
question3 = "Bạn là ai?"
prompt = "Hỏi:"+ question3 + '''
    Đáp:'''
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=100,
                             no_repeat_ngram_size=2,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như viết email hoặc gửi tin nhắn Tuy nhiên tôi có thể cung cấp cho bạn một số thông tin về bản thân của tôi Dưới đây là những gì tôi đã làm trong thời gian gần đây 1 Tôi đã tạo ra một tài khoản trên một nền tảng truyền thông xã hội để chia sẻ với bạn bè và người hâm mộ của mình về các chủ đề liên quan đến khoa học và công nghệ 2
Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như viết email hoặc gửi tin nhắn Tuy nhiên tôi có thể cung cấp cho bạn một số thông tin về bản thân của tôi Dưới đây là những gì tôi đã làm trong thời gian gần đây 1 Tôi đã tạo ra một tài khoản trên một nền tảng truyền thông xã hội để chia sẻ với bạn bè và người hâm mộ của mình về các chủ đề liên quan đến khoa học máy tính 2 Tôi
Hỏi:Bạn là ai?
    Đáp: Là một mô hình ngôn ngữ AI, tôi không có khả năng thực hiện các hoạt động cá nhân như viết email hoặc 

In [ ]:
question3 = "Viết bài tả con mèo"
prompt = "Hỏi:"+ question3 + '''
    Đáp:'''
# In ra kết quả
inputs = tokenizer( prompt, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                             max_new_tokens=300,
                             no_repeat_ngram_size=2,
                             num_beams=3,
                             num_return_sequences=3)
    for i in tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True):
      print(i)

Hỏi:Viết bài tả con mèo
    Đáp: Con mèo là một trong những động vật có vú phổ biến nhất trên thế giới Nó được biết đến với sự linh hoạt và khả năng thích nghi với môi trường sống của nó Nó có thể được tìm thấy ở nhiều khu vực khác nhau từ châu Á đến châu Âu và Nam Mỹ Chúng có một số đặc điểm độc đáo mà chúng ta sẽ đề cập ở đây Đầu tiên là màu sắc của chúng Mèo thường có màu nâu hoặc nâu đỏ hoặc màu xanh lá cây tùy thuộc vào độ sâu của màu của lông và da Chúng cũng có nhiều màu khác như màu đỏ và xanh dương và màu trắng và đen Chúng thường được gọi là mèo vì chúng có mùi thơm đặc biệt của họ Chúng là những người bạn thân thiết của con người và chúng cũng rất thân thiện với người khác Chúng rất dễ thương và rất vui vẻ với con cái chúng Chúng thích chơi với nhau và thích làm quen với những con vật khác chúng thường thích ăn và ăn những thứ như rau và rau tươi và các loại thực phẩm khác Một số loài mèo khác có lông dài và dài hơn và có những lông ngắn và ngắn hơn Chúng không cần phải ăn n